In [1]:
#!pip install simple_salesforce

In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [39]:
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [40]:
student_data=student_data[['ID_Student','StudentID','LastName', 'FirstName','MiddleName','BirthDate','Gender']]
student_data.head(10)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F
5,38,25006014,Poocharoen,Pariya,Mariette,None,F
6,39,25007528,Mantrala,Sriharitha,Patty,None,F
7,40,25003778,Kelly,Carly,Raelene,None,F
8,41,25003605,Sraha,Clementine,Kendall,None,F
9,42,25002056,Abdulrahim,Jawaher,Angela,None,F


In [42]:
#for index, row in student_data.iterrows():
#    record = {
#        'IDStudent__c': row['ID_Student'],
#        'StudentID__c': int(row['StudentID']),
#        'LastName__c':row['LastName'],
#        'Name':row['FirstName'],
#        'MiddleName__c':row['MiddleName'],
#        'BirthDate__c':row['BirthDate'],
#        'Gender__c':row['Gender']
#    try:
#        sf.Student__c.create(record)
#    except Exception as e:
#       print(e)

In [43]:
student_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT StudentID__c, Name FROM Student__c")
for row in data:
    rec = {
        'Student__c': row['Name'],
        'StudentID__c': row['StudentID__c']
    }
    student_lookup_list.append(rec)

In [44]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df

,Student__c,StudentID__c
0,Ilana,25003514
1,Robert,25006027
2,Chris,25006348
3,Cole,25005250
4,Undraa,25004097
5,Michael,25005443
6,Clementine,25003605
7,John,25003936
8,Katherine,25007185
9,Christopher,25002555


In [49]:
query = '''
    SELECT
        c.*
        ,s.StudentID
    FROM
        classparticipant c
        INNER JOIN student s
        ON c.ID_student = s.ID_student
'''
classparticipant_data_df = pd.read_sql(query, conn)
classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate,StudentID
0,1,33,1,2020-03-16,None,25004961
1,2,34,1,2020-03-16,None,25003514
2,3,35,1,2020-03-16,None,25005833
3,4,62,1,2020-03-16,None,25007334
4,5,36,1,2020-03-16,None,25002589


In [50]:
classparticipant_data_df.rename(columns={
    'StudentID':'StudentID__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    
}, inplace=True)

classparticipant_data_df

,ID_ClassParticipant,ID_Student,ID_Class,Start_Date__c,End_Date__c,StudentID__c
0,1,33,1,2020-03-16,None,25004961
1,2,34,1,2020-03-16,None,25003514
2,3,35,1,2020-03-16,None,25005833
3,4,62,1,2020-03-16,None,25007334
4,5,36,1,2020-03-16,None,25002589
5,6,37,1,2020-03-16,None,25007185
6,7,38,1,2020-03-16,None,25006014
7,8,39,1,2020-03-16,None,25007528
8,9,40,1,2020-03-16,None,25003778
9,10,41,1,2020-03-16,None,25003605


In [51]:
classparticipant_data_df = pd.merge(classparticipant_data_df, student_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,Start_Date__c,End_Date__c,StudentID__c,Student__c
0,1,33,1,2020-03-16,None,25004961,Heather
1,2,34,1,2020-03-16,None,25003514,Ilana
2,3,35,1,2020-03-16,None,25005833,Jessica
3,4,62,1,2020-03-16,None,25007334,Kevin
4,5,36,1,2020-03-16,None,25002589,Kandra


In [52]:
classparticipant_data_df
classparticipant_data_df['Start_Date__c'] = pd.to_datetime(classparticipant_data_df['Start_Date__c']).dt.date
classparticipant_data_df['End_Date__c'] = pd.to_datetime(classparticipant_data_df['End_Date__c']).dt.date

classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,Start_Date__c,End_Date__c,StudentID__c,Student__c
0,1,33,1,2020-03-16,NaT,25004961,Heather
1,2,34,1,2020-03-16,NaT,25003514,Ilana
2,3,35,1,2020-03-16,NaT,25005833,Jessica
3,4,62,1,2020-03-16,NaT,25007334,Kevin
4,5,36,1,2020-03-16,NaT,25002589,Kandra


In [69]:
classparticipant_data_df=classparticipant_data_df[['ID_ClassParticipant','ID_Student','ID_Class','Start_Date__c',
                                       'StudentID__c','Student__c']]
classparticipant_data_df

,ID_ClassParticipant,ID_Student,ID_Class,Start_Date__c,StudentID__c,Student__c
0,1,33,1,2020-03-16,25004961,Heather
1,2,34,1,2020-03-16,25003514,Ilana
2,3,35,1,2020-03-16,25005833,Jessica
3,4,62,1,2020-03-16,25007334,Kevin
4,5,36,1,2020-03-16,25002589,Kandra
5,6,37,1,2020-03-16,25007185,Katherine
6,7,38,1,2020-03-16,25006014,Pariya
7,8,39,1,2020-03-16,25007528,Sriharitha
8,9,40,1,2020-03-16,25003778,Carly
9,10,41,1,2020-03-16,25003605,Clementine


In [70]:
classparticipant_data_df_records = classparticipant_data_df.to_dict(orient='records')
classparticipant_data_df_records

[{'ID_ClassParticipant': 1,
  'ID_Student': 33,
  'ID_Class': 1,
  'Start_Date__c': datetime.date(2020, 3, 16),
  'StudentID__c': '25004961',
  'Student__c': 'Heather'},
 {'ID_ClassParticipant': 2,
  'ID_Student': 34,
  'ID_Class': 1,
  'Start_Date__c': datetime.date(2020, 3, 16),
  'StudentID__c': '25003514',
  'Student__c': 'Ilana'},
 {'ID_ClassParticipant': 3,
  'ID_Student': 35,
  'ID_Class': 1,
  'Start_Date__c': datetime.date(2020, 3, 16),
  'StudentID__c': '25005833',
  'Student__c': 'Jessica'},
 {'ID_ClassParticipant': 4,
  'ID_Student': 62,
  'ID_Class': 1,
  'Start_Date__c': datetime.date(2020, 3, 16),
  'StudentID__c': '25007334',
  'Student__c': 'Kevin'},
 {'ID_ClassParticipant': 5,
  'ID_Student': 36,
  'ID_Class': 1,
  'Start_Date__c': datetime.date(2020, 3, 16),
  'StudentID__c': '25002589',
  'Student__c': 'Kandra'},
 {'ID_ClassParticipant': 6,
  'ID_Student': 37,
  'ID_Class': 1,
  'Start_Date__c': datetime.date(2020, 3, 16),
  'StudentID__c': '25007185',
  'Student__c

In [77]:
for rec in classparticipant_data_df_records:
 
    record = {
        'Name': rec['Student__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        
    }
    
    try:
        sf.ClassParticipant__c.create(record)
    except Exception as e:
        print(e)

Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__c/. Response content: [{'message': 'Required fields are missing: [ID_Student__c, ID_Class__c]', 'errorCode': 'REQUIRED_FIELD_MISSING', 'fields': ['ID_Student__c', 'ID_Class__c']}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__c/. Response content: [{'message': 'Required fields are missing: [ID_Student__c, ID_Class__c]', 'errorCode': 'REQUIRED_FIELD_MISSING', 'fields': ['ID_Student__c', 'ID_Class__c']}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__c/. Response content: [{'message': 'Required fields are missing: [ID_Student__c, ID_Class__c]', 'errorCode': 'REQUIRED_FIELD_MISSING', 'fields': ['ID_Student__c', 'ID_Class__c']}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__c/. Response content: [{'message': 'Required fields are missing: [ID_Student__c, ID_